# 1. Import libraries

In [1]:
import os
import numpy as np
import pandas as pd

# 2. Set path

In [2]:
path = r'/Users/aibee/Desktop/CareerFoundry/4. Python Fundamentals for Data Analysts/24-05-2022 Instacart Basket Analysis'

# 3. Import Data

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged.pkl'))

In [4]:
df = ords_prods_merge

# 4. Aggregating data

In [5]:
#to aggregate data, start by grouping then aggregating
df.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


### The entire dataframe includes more departments than the subset. Also, the range of the mean for the entire dataframe is 15 to 30. While, the range of mean for the subset is 11 to 19.

# 5. creating a loyalty flag

In [6]:
# creating a new column for loyalty
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [7]:
#removing maximum number of roles for display
pd.options.display.max_rows = None

In [8]:
ords_prods_merge.head(100)

,order_id,user_id,order_number,orders_day_of_week,order_time_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,max_order
0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,Soda,77,7,9.0,both,10
1,2398795,1,2,3,7,15.0,196.0,1.0,1.0,Soda,77,7,9.0,both,10
2,473747,1,3,3,12,21.0,196.0,1.0,1.0,Soda,77,7,9.0,both,10
3,2254736,1,4,4,7,29.0,196.0,1.0,1.0,Soda,77,7,9.0,both,10
4,431534,1,5,4,15,28.0,196.0,1.0,1.0,Soda,77,7,9.0,both,10
5,3367565,1,6,2,7,19.0,196.0,1.0,1.0,Soda,77,7,9.0,both,10
6,550135,1,7,1,9,20.0,196.0,1.0,1.0,Soda,77,7,9.0,both,10
7,3108588,1,8,1,14,14.0,196.0,2.0,1.0,Soda,77,7,9.0,both,10
8,2295261,1,9,1,16,0.0,196.0,4.0,1.0,Soda,77,7,9.0,both,10
9,2550362,1,10,4,8,30.0,196.0,1.0,1.0,Soda,77,7,9.0,both,10


In [9]:
# using the loc() function to group the customers
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [10]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [11]:
#check the average price of product purchase by the customer groups
ords_prods_merge.groupby('loyalty_flag').agg({'prices':['mean','max','min']})


prices              
                       mean      max  min
loyalty_flag                             
Loyal customer    10.386336  99999.0  1.0
New customer      13.294670  99999.0  1.0
Regular customer  12.495717  99999.0  1.0

### On the average, loyal customers purchase the cheapest products, followed by regular and new customers

# 6. Spender Flag

In [12]:
#create a new column for average prices for each user
ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [13]:
#deriving new column with loc
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spender_flag'] = 'High spender'
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spender_flag'] = 'Low spender'

In [14]:
ords_prods_merge['spender_flag'].value_counts(dropna = False)

Low spender     31770742
High spender      634117
Name: spender_flag, dtype: int64

# 7. Frequency Flag

In [15]:
#create a new column according to median since last order
ords_prods_merge['median_order_duration'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [16]:
#deriving new column with loc
ords_prods_merge.loc[ords_prods_merge['median_order_duration'] > 20, 'frequency_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_order_duration'] > 10) & (ords_prods_merge['median_order_duration'] <= 20), 'frequency_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_order_duration'] < 10, 'frequency_flag'] = 'Frequent customer'

In [17]:
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

Frequent customer        20287988
Regular customer          7208564
Non-frequent customer     3636437
NaN                       1271870
Name: frequency_flag, dtype: int64

### The frequent customers are the most customers purchasing from the store.